# Sentinel 2 Collection

In [1]:
from geetools import collection, tools, ui

In [2]:
import ee

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [3]:
def print_values(image):
    print(tools.image.get_value(image, image.geometry().centroid(), 30, side='client'))

## Sentinel 2 SR

In [4]:
s2sr = collection.Sentinel2SR()

In [5]:
collection.info(s2sr)

{'algorithms': {'hollstein': <function geetools.cloud_mask.apply_hollstein(image, options=('cloud', 'snow', 'shadow', 'water', 'cirrus'), aerosol='B1', blue='B2', green='B3', red_edge1='B5', red_edge2='B6', red_edge3='B7', red_edge4='B8A', water_vapor='B9', cirrus='B10', swir='B11')>,
  'scl_masks': <bound method Sentinel2.SCL_masks of <geetools.collection.sentinel.Sentinel2 object at 0x7faafabb0e10>>},
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'red_edge_1',
  'red_edge_2',
  'red_edge_3',
  'nir',
  'red_edge_4',
  'water_vapor',
  'swir',
  'swir2',
  'aerosol_thickness',
  'water_vapor_pressure',
  'scene_classification_map',
  'qa10',
  'qa20',
  'qa60'],
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL',
  'QA10',
  'QA20',
  'QA60'],
 'bit_bands': ['QA10', 'QA20', 'QA60'],
 'classification_bands': ['SCL'],
 'cloud_cover': 'CLOUD_COVERAGE_ASSESSMENT',
 'ee_collection': <ee.imagecol

In [6]:
s2sr_i = ee.Image(s2sr.collection.first())

In [7]:
for band in s2sr.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 red_edge_1
B6 red_edge_2
B7 red_edge_3
B8 nir
B8A red_edge_4
B9 water_vapor
B11 swir
B12 swir2
AOT aerosol_thickness
WVP water_vapor_pressure
SCL scene_classification_map
QA10 qa10
QA20 qa20
QA60 qa60


## Rename

In [8]:
s2sr_i = s2sr_i.addBands(ee.Image.constant(0).rename('test'))

In [9]:
s2sr_renamed = s2sr.rename(s2sr_i)

In [10]:
ui.eprint(s2sr_renamed.bandNames())

[ 'aerosol',
  'blue',
  'green',
  'red',
  'red_edge_1',
  'red_edge_2',
  'red_edge_3',
  'nir',
  'red_edge_4',
  'water_vapor',
  'swir',
  'swir2',
  'aerosol_thickness',
  'water_vapor_pressure',
  'scene_classification_map',
  'TCI_R',
  'TCI_G',
  'TCI_B',
  'MSK_CLDPRB',
  'MSK_SNWPRB',
  'qa10',
  'qa20',
  'qa60',
  'test']



## Rename collection

In [11]:
s2col = s2sr.collection

In [12]:
s2col_renamed = s2col.map(lambda img: s2sr.rename(img))

In [13]:
tools.imagecollection.get_values(s2col_renamed.limit(2), ee.Image(s2col.first()).geometry().centroid(), 20, side='client')

{'20190209T000239_20190209T000240_T55GDQ': {'MSK_CLDPRB': 0.0,
  'MSK_SNWPRB': 72.0,
  'TCI_B': 255.0,
  'TCI_G': 255.0,
  'TCI_R': 255.0,
  'aerosol': 9145.0,
  'aerosol_thickness': 204.0,
  'blue': 9328.0,
  'green': 8232.0,
  'nir': 8039.0,
  'qa10': 0.0,
  'qa20': 0.0,
  'qa60': 1024.0,
  'red': 7610.0,
  'red_edge_1': 7892.0,
  'red_edge_2': 7546.0,
  'red_edge_3': 7286.0,
  'red_edge_4': 6960.0,
  'scene_classification_map': 8.0,
  'swir': 2408.0,
  'swir2': 2471.0,
  'water_vapor': 14617.0,
  'water_vapor_pressure': 890.0},
 '20190209T000239_20190209T000240_T55GDR': {'MSK_CLDPRB': None,
  'MSK_SNWPRB': None,
  'TCI_B': None,
  'TCI_G': None,
  'TCI_R': None,
  'aerosol': None,
  'aerosol_thickness': None,
  'blue': None,
  'green': None,
  'nir': None,
  'qa10': None,
  'qa20': None,
  'qa60': None,
  'red': None,
  'red_edge_1': None,
  'red_edge_2': None,
  'red_edge_3': None,
  'red_edge_4': None,
  'scene_classification_map': None,
  'swir': None,
  'swir2': None,
  'water_v

## SCL Mask

In [14]:
mask = s2sr.SCL_masks(s2sr_i)

## Visualization

In [15]:
Map = ui.Map()

In [16]:
Map.show()

Map(basemap={'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a…

### renamed image

In [17]:
p = ee.Geometry.Point([-71, -41])

In [18]:
s2col_renamed = s2col_renamed.filterBounds(p)

In [19]:
s2sr_renamed = s2col_renamed.first()

In [20]:
vis = s2sr.visualization('NSR', True)

In [21]:
Map.addLayer(s2sr_renamed, vis, 'S2 SR Renamed NSR')

In [22]:
Map.centerObject(s2sr_renamed)

## SCL

In [23]:
visSCL = s2sr.visualization('SCL', True)

In [24]:
Map.addLayer(s2sr_renamed, visSCL, 'SCL')

## Masks

In [25]:
s2sr.bit_options(True)

{'qa60': ['cloud', 'cirrus']}

In [26]:
masked = s2sr.apply_mask(s2sr_renamed, 'qa60', renamed=True)

In [27]:
Map.addLayer(masked, vis, 'Masked')

## Hollstein

In [28]:
s2toa = collection.Sentinel2TOA()

In [29]:
i = ee.Image(s2toa.collection.filterBounds(p).filterMetadata(s2toa.cloud_cover, 'less_than', 50).first())

In [30]:
Map.addLayer(i, s2toa.visualization('NSR'), 'TOA')

In [31]:
holl = s2toa.algorithms['hollstein']

In [32]:
help(holl)

Help on function apply_hollstein in module geetools.cloud_mask:

apply_hollstein(image, options=('cloud', 'snow', 'shadow', 'water', 'cirrus'), aerosol='B1', blue='B2', green='B3', red_edge1='B5', red_edge2='B6', red_edge3='B7', red_edge4='B8A', water_vapor='B9', cirrus='B10', swir='B11')
    Apply Hollstein mask



In [35]:
iholl = holl(i, ['cloud', 'shadow', 'snow'])

In [36]:
Map.addLayer(iholl, s2toa.visualization('NSR'), 'Masked TOA')